# Time-Series Visualization using bokeh/plotly - Neerja Doshi
For this assignment, Bokeh has been used as the package for creating plots.

In [1]:
# import packages
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
import datetime
from bokeh.charts import HeatMap
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar

/Users/neerjadoshi/anaconda2/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


In [2]:
# Load the data
data = pd.read_csv('Monthly_Property_Crime_2005_to_2015.csv', parse_dates=['Date'])
data.head()

,Date,Category,IncidntNum
0,2014-02-01,BURGLARY,506
1,2007-02-01,VANDALISM,531
2,2012-07-01,BURGLARY,522
3,2013-07-01,LARCENY/THEFT,3318
4,2010-08-01,VANDALISM,694


In [3]:
data.Date.min(), data.Date.max()

(Timestamp('2005-01-01 00:00:00'), Timestamp('2015-12-01 00:00:00'))

In [4]:
data.Category.value_counts()

STOLEN PROPERTY    132
VEHICLE THEFT      132
VANDALISM          132
LARCENY/THEFT      132
ARSON              132
BURGLARY           132
Name: Category, dtype: int64

In [5]:
data['Year'] = data.Date.apply(lambda x: x.year)
data['Month'] = data.Date.apply(lambda x: x.month)
data.head()

,Date,Category,IncidntNum,Year,Month
0,2014-02-01,BURGLARY,506,2014,2
1,2007-02-01,VANDALISM,531,2007,2
2,2012-07-01,BURGLARY,522,2012,7
3,2013-07-01,LARCENY/THEFT,3318,2013,7
4,2010-08-01,VANDALISM,694,2010,8


In [6]:
burglary = data[data.Category == 'BURGLARY'].sort_values(['Date'])
stolen_property = data[data.Category == 'STOLEN PROPERTY'].sort_values(['Date'])
vehicle_theft = data[data.Category == 'VEHICLE THEFT'].sort_values(['Date'])
vandalism = data[data.Category == 'VANDALISM'].sort_values(['Date'])
larceny = data[data.Category == 'LARCENY/THEFT'].sort_values(['Date'])
arson = data[data.Category == 'ARSON'].sort_values(['Date'])
arson.head()

,Date,Category,IncidntNum,Year,Month
198,2005-01-01,ARSON,21,2005,1
215,2005-02-01,ARSON,17,2005,2
36,2005-03-01,ARSON,19,2005,3
313,2005-04-01,ARSON,24,2005,4
261,2005-05-01,ARSON,13,2005,5


In [7]:
output_notebook()

Loading BokehJS ...

### Bar Chart
This is used to analyse the average crimes per month. All the months have between 600-800 average crimes, with February being the least.

In [8]:
temp_df = data.groupby(['Month']).mean().reset_index()
temp_df.head()

,Month,IncidntNum,Year
0,1,702.560606,2010.0
1,2,620.409091,2010.0
2,3,707.712121,2010.0
3,4,700.060606,2010.0
4,5,708.378788,2010.0


In [9]:
TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom,tap"
p = figure(plot_height=350,
#     plot_width=1000,
    title="Average Number of Crimes by Month",
    tools=TOOLS,
    toolbar_location='above')

p.vbar(x=temp_df.Month, top=temp_df.IncidntNum, width=0.9)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.xaxis.axis_label = 'Month'
p.yaxis.axis_label = 'Average Crimes'
p.select_one(HoverTool).tooltips = [
    ('month', '@x'),
    ('Number of crimes', '@top'),
]
output_file("barchart.html", title="barchart")
show(p)

### Line Chart
This plot shows the trend in number of crimes over the years. It can be seen that the crime rate decreased from 2005-2010, with 2010 having the lowest crime rate. From then on, it has kept increasing steadily with 2015 having the highest number of crimes.

In [10]:
temp_df = data.groupby(['Year']).sum().reset_index()
temp_df.head()

,Year,IncidntNum,Month
0,2005,58368,468
1,2006,50150,468
2,2007,46023,468
3,2008,45647,468
4,2009,44633,468


In [11]:
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
p = figure(title="Year-wise total number of crimes", y_axis_type="linear", plot_height = 400,
           tools = TOOLS, plot_width = 800)
p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Total Crimes'
p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

p.line(temp_df.Year, temp_df.IncidntNum,line_color="purple", line_width = 3)
p.select_one(HoverTool).tooltips = [
    ('year', '@x'),
    ('Number of crimes', '@y'),
]

output_file("line_chart.html", title="Line Chart")
show(p)

### Stacked bar chart
This chart explores the distribution of crimes among the various categories over the years. In particular, larceny/theft are the most frequently occuring crimes, while stolen property occur the least. 2005 saw a high number of vehicle thefts, which reduced quite a bit subsequently.

In [12]:
wide = data.pivot(index='Date', columns='Category', values='IncidntNum')
wide.reset_index(inplace=True)
wide['Year'] = wide.Date.apply(lambda x: x.year)
wide['Month'] = wide.Date.apply(lambda x: x.month)

temp_df = wide.groupby(['Year']).sum().reset_index()
temp_df.head()
cats = ['ARSON','BURGLARY','LARCENY/THEFT','STOLEN PROPERTY','VANDALISM','VEHICLE THEFT'] 
temp_df.drop(['Month'], axis = 1, inplace=True)
temp_df.head()

Category,Year,ARSON,BURGLARY,LARCENY/THEFT,STOLEN PROPERTY,VANDALISM,VEHICLE THEFT
0,2005,231,7071,25319,540,7013,18194
1,2006,240,7004,27352,575,7688,7291
2,2007,246,5454,25770,527,7566,6460
3,2008,248,5679,25807,518,7342,6053
4,2009,222,5379,25585,660,7604,5183


In [13]:
TOOLS = "save,pan,box_zoom,reset,wheel_zoom,tap"

source = ColumnDataSource(data=temp_df)
p = figure( plot_width=800, title="Category wise count of crimes by year",toolbar_location='above', tools=TOOLS)
colors = brewer['Dark2'][6]

p.vbar_stack(cats, x='Year', width=0.9, color=colors, source=source,
             legend=[value(x) for x in cats])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Total Crimes'
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

output_file("stacked_bar.html", title="Stacked Bar Chart")

show(p)

### Heat Map
The heat map shows the total number of crimes by month and year, with darker colours represnting higher number of crimes. Months in 2015 have the highest total while the least are in the some months in 2009 and 2010.

In [14]:
temp_df = data.groupby(['Year', 'Month']).sum().reset_index()
# temp_df['Month_Category'] = pd.concat([temp_df['Month'], temp_df['Category']], axis = 1)
temp_df.head()

,Year,Month,IncidntNum
0,2005,1,4991
1,2005,2,4166
2,2005,3,4815
3,2005,4,4711
4,2005,5,5120


In [15]:
TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom,tap"
hm = figure(title="Month-Year wise crimes", tools=TOOLS, toolbar_location='above')

source = ColumnDataSource(temp_df)
colors = brewer['BuGn'][9]
colors = colors[::-1]
mapper = LinearColorMapper(
    palette=colors, low=temp_df.IncidntNum.min(), high=temp_df.IncidntNum.max())
hm.rect(x="Year", y="Month",width=2,height=1,source = source,  
    fill_color={
        'field': 'IncidntNum',
        'transform': mapper
    },
    line_color=None)
color_bar = ColorBar(
    color_mapper=mapper,
    major_label_text_font_size="10pt",
    ticker=BasicTicker(desired_num_ticks=len(colors)),
    formatter=PrintfTickFormatter(),
    label_standoff=6,
    border_line_color=None,
    location=(0, 0))

hm.add_layout(color_bar, 'right')
hm.xaxis.axis_label = 'Year'
hm.yaxis.axis_label = 'Month'
hm.select_one(HoverTool).tooltips = [
    ('Year', '@Year'),('Month', '@Month'), ('Number of Crimes', '@IncidntNum')
]

output_file("heatmap.html", title="Heat Map")

show(hm)  # open a browser

### Multiline Plot
This plot shows the distribution of crimes across categories over the years. This plot shows information similar to the stacked bar chart, except that here it is easier to note that arson and property theft amount to almost the same amount of crimes every year. Similarly, vehicle theft(except for 2005), vandalism and burglary have very similar patterns. Only larceny/theft is increasing with every passing year and its count is much higher than any of the other types of crimes.

In [17]:
TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
p = figure(title="Category-wise crimes through Time", y_axis_type="linear",x_axis_type='datetime', tools = TOOLS)

p.line(burglary['Date'], burglary.IncidntNum, legend="burglary", line_color="purple", line_width = 3)
p.line(stolen_property['Date'], stolen_property.IncidntNum, legend="stolen_property", line_color="blue", line_width = 3)

p.line(vehicle_theft['Date'], vehicle_theft.IncidntNum, legend="vehicle_theft", line_color = 'coral', line_width = 3)

p.line(larceny['Date'], larceny.IncidntNum, legend="larceny", line_color='green', line_width = 3)

p.line(vandalism['Date'], vandalism.IncidntNum, legend="vandalism", line_color="gold", line_width = 3)

p.line(arson['Date'], arson.IncidntNum, legend="arson", line_color="magenta",line_width = 3)

p.legend.location = "top_left"

p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Count'

output_file("multiline_plot.html", title="Multi Line Plot")

show(p)  # open a browser